In [1]:
import os  #finds our current directory path

In [2]:
#set some variables
STRUCTURE= ['RS', 'TSA']

In [ ]:
# for i in STRUCTURE:
#     for index in range (1,11,1):
#         structure_folder = f'/bp1/mulholland/zn22275/MalL_{i}/output_2/run_{index}'
#         !mkdir "$structure_folder"

Strip ligand from equilibrium files

In [ ]:
# Function to send jobs for stripping
def runjob_strip(structure,index):
    print(f"submitted MD_{index}_{structure}")
    temp = f"""#!/bin/bash
#SBATCH --account=CHEM021482
#SBATCH --partition test  ##Using test partition b/c v. slow queue for compute  
#SBATCH --job-name=strip_{structure}
#SBATCH --output   strip_{structure}
#SBATCH --time=1:00:00       # Walltime 1 hour max on this partition
#SBATCH --nodes=12            # number of tasks was 12 
#SBATCH --ntasks-per-node=1 # number of tasks per node
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G

# 1. Load module(s)
module load apps/amber/24  

# 2. Set directories
cd {os.getcwd()}

#3.autoimage .rst7 then strip ligand
for rep in $(seq 100 20 500)
do 
    cat > delete.in <<EOF
parm /bp1/mulholland/zn22275/300K_MalL_{structure}/MalL_{structure}.prmtop
trajin /bp1/mulholland/zn22275/300K_MalL_{structure}/run_{index}/run_md${{rep}}.rst7
strip :562,563,564
autoimage
trajout /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md${{rep}}.rst7 ncrestart
go
EOF
    cpptraj < delete.in &> delete.out
done
/       
"""
    with open(f"MD.sh", "w") as f:
        f.writelines(temp)

    !sbatch --exclude=bp1-gpu[007,035,030] MD.sh

In [13]:
# repeats=[2]
for structure in STRUCTURE:         
    for index in range(1,11,1):
        runjob_strip(structure=structure,
            index=str(index))

submitted MD_1_RS
Submitted batch job 10854821


submitted MD_2_RS
Submitted batch job 10854822
submitted MD_3_RS
Submitted batch job 10854823
submitted MD_4_RS
Submitted batch job 10854824
submitted MD_5_RS
Submitted batch job 10854825
submitted MD_6_RS
Submitted batch job 10854826
submitted MD_7_RS
Submitted batch job 10854827
submitted MD_8_RS
Submitted batch job 10854828
submitted MD_9_RS
Submitted batch job 10854829
submitted MD_10_RS
Submitted batch job 10854830
submitted MD_1_TSA
Submitted batch job 10854831
submitted MD_2_TSA
Submitted batch job 10854832
submitted MD_3_TSA
Submitted batch job 10854833
submitted MD_4_TSA
Submitted batch job 10854834
submitted MD_5_TSA
Submitted batch job 10854835
submitted MD_6_TSA
Submitted batch job 10854836
submitted MD_7_TSA
Submitted batch job 10854837
submitted MD_8_TSA
Submitted batch job 10854838
submitted MD_9_TSA
Submitted batch job 10854839
submitted MD_10_TSA
Submitted batch job 10854840


Running D-NEMD simulations

In [ ]:
#writing AMBER .in files for md

#no restraint


for structure in STRUCTURE:


    filename = f"/user/work/zn22275/MalL_{structure}/output/{structure}"
    print (filename)


    #Make .in files    (write what each line means)

    md5 = f"""5ns NVT MD production (weak coupling of 10 ps)

&cntrl 
    imin                = 0,       

    irest               = 1,       

    ntx                 = 5,       

    iwrap               = 1,       

    nstlim              = 2500000,  

    dt                  = 0.002,   

    ntpr                = 500, 

    ntwx                = 500, 

    ioutfm              = 1, 

    ntwr                = 50000,

    ntc                 = 2, 

    ntf                 = 2, 

    cut                 = 8.0,

    ntb                 = 1, 

    ntp                 = 0,

    ntt                 = 1, 

    tautp               = 10.0, 

    temp0               = 300.0,      !temperature which we will be changing

    nmropt              = 1,

/

&wt type='DUMPFREQ', istep1=500 /

&wt type='END' / 

/

"""    

    with open(f"{filename}_md5.in", "w") as f:

        f.writelines(md5)

In [4]:
def runjob_MD(structure,index):
    print(f"submitted MD_{index}_{structure}")
    temp = f"""#!/bin/bash
#SBATCH --account=CHEM021482
#SBATCH --gres=gpu:1
#SBATCH --partition gpu
#SBATCH --job-name=MD_{index}_{structure}
#SBATCH --output   MD_{index}_{structure}
#SBATCH --time=3-00:00:00  
#SBATCH --mem=12G

# 1. Load module(s)
module load apps/amber/24.tools.24

# 2. Set directories
cd {os.getcwd()}

topology_file= /bp1/mulholland/zn22275/MalL_{structure}/output/stripped_MalL_{structure}.prmtop
input_filepath= /bp1/mulholland/zn22275/MalL_{structure}/output/{structure}_md5.in
md_rst7_input= /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md
output_filepath= /user/work/zn22275/{structure}_dnemd/run_{index} 

# 3. Run MD job(s)  

for x in $(seq 100 20 500)
do
    #autoimage .rst7 file to avoid problems
    cat > image.in <<EOF
parm /bp1/mulholland/zn22275/MalL_{structure}/output/stripped_MalL_{structure}.prmtop 
trajin /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md${{x}}.rst7  
autoimage
trajout /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md${{x}}.rst7 ncrestart
go
EOF
    cpptraj < image.in &> image.out       
    #run md jobs
    pmemd.cuda  -O -i /bp1/mulholland/zn22275/MalL_{structure}/output/{structure}_md5.in -o /user/work/zn22275/{structure}_dnemd/run_{index}/dnemd_${{x}}.log -p /bp1/mulholland/zn22275/MalL_{structure}/output/stripped_MalL_{structure}.prmtop -c /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md${{x}}.rst7 -x /user/work/zn22275/{structure}_dnemd/run_{index}/dnemd_${{x}}.nc -r /user/work/zn22275/{structure}_dnemd/run_{index}/dnemd_${{x}}.rst7

    cat > strip.in <<EOF
parm /bp1/mulholland/zn22275/MalL_{structure}/output/stripped_MalL_{structure}.prmtop
trajin /user/work/zn22275/{structure}_dnemd/run_{index}/dnemd_${{x}}.nc
autoimage
strip @Na+
strip @Cl-
strip :WAT outprefix dry
outtraj /user/work/zn22275/{structure}_dnemd/run_{index}/dry_md${{x}}.nc netcdf nobox
go
EOF
    cpptraj < strip.in &> strip.out

    #remove old nc after striping
    rm /user/work/zn22275/{structure}_dnemd/run_{index}/dnemd_${{x}}.nc
done
/       
"""

    with open(f"MD.sh", "w") as f:
        f.writelines(temp)

    !sbatch --exclude=bp1-gpu[007,035,030,024] MD.sh

In [5]:
for structure in STRUCTURE:       
    for index in range(1,11,1):
        runjob_MD(structure=structure,
            index=str(index))

submitted MD_1_RS
Submitted batch job 10856166
submitted MD_2_RS
Submitted batch job 10856167
submitted MD_3_RS
Submitted batch job 10856168
submitted MD_4_RS
Submitted batch job 10856169
submitted MD_5_RS
Submitted batch job 10856170
submitted MD_6_RS
Submitted batch job 10856171
submitted MD_7_RS
Submitted batch job 10856172
submitted MD_8_RS
Submitted batch job 10856173
submitted MD_9_RS
Submitted batch job 10856174
submitted MD_10_RS
Submitted batch job 10856175
submitted MD_1_TSA
Submitted batch job 10856176
submitted MD_2_TSA
Submitted batch job 10856177
submitted MD_3_TSA
Submitted batch job 10856178
submitted MD_4_TSA
Submitted batch job 10856179
submitted MD_5_TSA
Submitted batch job 10856180
submitted MD_6_TSA
Submitted batch job 10856181
submitted MD_7_TSA
Submitted batch job 10856182
submitted MD_8_TSA
Submitted batch job 10856183
submitted MD_9_TSA
Submitted batch job 10856184
submitted MD_10_TSA
Submitted batch job 10856185


Make pdb files for equilibrium and D-NEMD

In [ ]:
# Make pdb - equilibrium
dry_pdb = f"""#!/bin/bash
#SBATCH --account=CHEM021482
#SBATCH --partition test  ##Using test partition b/c v. slow queue for compute  
#SBATCH --job-name=make_dry_pdb
#SBATCH --output   make_dry_pdb
#SBATCH --time=1:00:00       # Walltime 1 hour max on this partition
#SBATCH --nodes=1            # number of tasks was 12 
#SBATCH --ntasks-per-node=1 # number of tasks per node
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G

module load apps/amber/24.tools.24

cd {os.getcwd()}

source /user/work/zn22275/anaconda3/etc/profile.d/conda.sh

conda activate /user/work/zn22275/anaconda3/envs/dnemd

python make_pdb.py

"""

with open(f"submit_python_script.sh","w") as f:
    f.writelines(dry_pdb)

In [4]:
!sbatch submit_python_script.sh

Submitted batch job 11522811


In [ ]:
# Make pdbs - D-NEMD
dnemd_pdb = f"""#!/bin/bash
#SBATCH --account=CHEM021482
#SBATCH --partition test  ##Using test partition b/c v. slow queue for compute  
#SBATCH --job-name=make_dnemd_pdb
#SBATCH --output   make_dnemd_pdb
#SBATCH --time=1:00:00       # Walltime 1 hour max on this partition
#SBATCH --nodes=1            # number of tasks was 12 
#SBATCH --ntasks-per-node=1 # number of tasks per node
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G

module load apps/amber/24.tools.24

cd {os.getcwd()}

source /user/work/zn22275/anaconda3/etc/profile.d/conda.sh

conda activate /user/work/zn22275/anaconda3/envs/dnemd

python dnemd_pdb.py

"""

with open(f"submit_python_script2.sh","w") as f:
    f.writelines(dnemd_pdb)

In [8]:
!sbatch submit_python_script2.sh

Submitted batch job 11223103


In [ ]:
# any missed pdb points
dnemd_tsa_pdb = f"""#!/bin/bash
#SBATCH --account=CHEM021482
#SBATCH --partition test  ##Using test partition b/c v. slow queue for compute  
#SBATCH --job-name=make_dnemd_pdb_tsa
#SBATCH --output   make_dnemd_pdb_tsa
#SBATCH --time=1:00:00       # Walltime 1 hour max on this partition
#SBATCH --nodes=1            # number of tasks was 12 
#SBATCH --ntasks-per-node=1 # number of tasks per node
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G

module load apps/amber/24.tools.24

cd {os.getcwd()}

source /user/work/zn22275/anaconda3/etc/profile.d/conda.sh

conda activate /user/work/zn22275/anaconda3/envs/dnemd

python dnemd_tsa_pdb.py

"""

with open(f"submit_python_script3.sh","w") as f:
    f.writelines(dnemd_tsa_pdb)

In [16]:
!sbatch submit_python_script3.sh

Submitted batch job 11226065


Displacement of D-NEMD from equilibrium

In [4]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import numpy as np
import matplotlib.pyplot as plt

STRUCTURE= ['RS','TSA']
runs = [1,2,3,4,5,6,7,8,9,10] 
times = [i for i in range(100, 500, 20)]

/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
# 2 sig Eq-DNEMD
for structure in STRUCTURE:
# Load the base reference C-alpha structure
    base_ref = mda.Universe(f'/user/work/zn22275/{structure}_dry_pdb/run_1/dry_md100_0.pdb')
    base_ca = base_ref.select_atoms('name CA')

# Create a new Universe object with only the C-alpha atoms
    ca_universe = mda.Merge(base_ca)

# Save the positions of base C-alpha atoms to a file
    np.savetxt("base_ca.txt", base_ca.positions)


    time_points =  [0, 2, 5, 7, 10, 12, 25, 50, 125, 150, 200, 250, 500, 750, 1000, 1250, 2500] 


    for tp in time_points:
        vectors = []
        for run in runs:
            for time in times:
                try:
                    ref = mda.Universe(f'/user/work/zn22275/{structure}_dry_pdb/run_{run}/dry_md{time}_{tp}.pdb')
                    other = mda.Universe(f'/user/work/zn22275/{structure}_dnemd_pdb/run_{run}/dnemd{time}_{tp}.pdb')
                    mda.analysis.align.alignto(other, ref, select='name CA')
                    ca_ref = ref.select_atoms('name CA').positions
                    ca_other = other.select_atoms('name CA').positions
                    diff = ca_other - ca_ref
                    vectors.append(diff)
                except Exception as e:
                    print(f"Error processing {run}/{time}: {e}")
                    continue

        avg_vectors = np.mean(np.array(vectors), axis=0)

        # Calculate stats
        avg_disp = np.linalg.norm(avg_vectors, axis=1)
        sd = np.std(vectors, axis=0)
        se = sd / np.sqrt(len(vectors))
        avg_x = avg_vectors[:, 0]
        avg_y = avg_vectors[:, 1]
        avg_z = avg_vectors[:, 2]

        # SE of magnitude
        se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))

        adjusted_avg_disp = np.where(np.abs(avg_disp) - 2*se_mag > 0, avg_disp, 0)
        adjusted_avg_vectors = np.where(np.abs(avg_vectors) - 2*se > 0, avg_vectors, 0)

        # # Write summary stats
        #     with open(f"vec_norm_stats_{tp}.txt", 'w') as f:
        #         for i in range(len(avg_vectors)):
        #             f.write(f"{adjusted_avg_vectors[i][0]}\t{adjusted_avg_vectors[i][1]}\t{adjusted_avg_vectors[i][2]}\t{adjusted_avg_disp[i]}\t{se[i][0]}\t{se[i][1]}\t{se[i][2]}\t{se_mag[i]}\n")

        # Write summary stats
        with open(f"/user/work/zn22275/dnemd_vec_{structure}/dnemd_stats/{structure}_vec_norm_stats_2sig_{tp}.txt", 'w') as f: # change file name
           # Write header
            f.write("#" * 128 + "\n")
            f.write("#                 Average Displacement                Sample Size                                   SE (1SD)                SE (1SD) of              SE (2SD)                SE (2SD) of       #\n")
            f.write("#              --------------------------       ---------------------      Average        --------------------------          Average        --------------------------          Average       #\n")
            f.write("#       CA     x-axis    y-axis    z-axis       x         y         z    Displacement     x-axis    y-axis    z-axis    Displacement vector  x-axis    y-axis    z-axis    Displacement vector #\n")
            f.write("#" * 128 + "\n")

            # Write data
            for i in range(len(avg_vectors)):
                f.write(f"{i+1:>9} {avg_vectors[i][0]:>8.3f} {avg_vectors[i][1]:>8.3f} {avg_vectors[i][2]:>8.3f} ") # removed adjusted vectors
                f.write(f"{len(vectors):>8} {len(vectors):>8} {len(vectors):>8} {avg_disp[i]:>12.3f} ") #removed adjusted displacement
                f.write(f"{se[i][0]:>10.3f} {se[i][1]:>10.3f} {se[i][2]:>10.3f} {se_mag[i]:>12.3f} ")
                f.write(f"{2*se[i][0]:>10.3f} {2*se[i][1]:>10.3f} {2*se[i][2]:>10.3f} {2*se_mag[i]:>12.3f}\n")


        # # Map the norms of the C-alpha vectors onto the C-alpha atoms in the new Universe object
        # for atom, bfactor in zip(ca_universe.atoms, adjusted_avg_disp):
        #     #atom.tempfactors = bfactor
        #     #print(bfactor)
        #     # with open(f"/user/work/zn22275/{structure}_vec/{structure}_vec_norm_{tp}.txt", 'w') as file:
        #     #     file.write(bfactor)

        # # ca_universe.atoms.tempfactors = adjusted_avg_disp
    
        # # Write the C-alpha-only structure with mapped B-factors
        # pdb_filename = f'/user/work/zn22275/{structure}_vec/{structure}_vec_norm_{tp}.pdb'
        # ca_universe.atoms.write(pdb_filename)

        with mda.Writer("temp_ca.pdb", ca_universe.atoms.n_atoms) as w: 
            w.write(ca_universe)
   
        # Now read it back and set the B-factors
        new_universe = mda.Universe("temp_ca.pdb")
        new_universe.add_TopologyAttr('tempfactors')  # Add the B-factor attribute explicitly
        new_universe.atoms.tempfactors = adjusted_avg_disp
   
        # Write the final PDB with B-factors
        pdb_filename = f'/user/work/zn22275/dnemd_vec_{structure}/{structure}_vec_norm_2sig_{tp}.pdb'
        with mda.Writer(pdb_filename) as w:
            w.write(new_universe)


/tmp/ipykernel_2112200/3774024314.py:44: RuntimeWarning: divide by zero encountered in divide
  se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))
/tmp/ipykernel_2112200/3774024314.py:44: RuntimeWarning: invalid value encountered in multiply
  se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1124: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:429: UserWarning: 1 

In [ ]:
# 0 D-NEMD pdb
def initial_pdb(structure,index,rep):
    initial_pdb = f"""
    parm /bp1/mulholland/zn22275/MalL_{structure}/output/stripped_MalL_{structure}.prmtop 
    trajin /bp1/mulholland/zn22275/MalL_{structure}/output_2/run_{index}/stripped_run_md{rep}.rst7 
    autoimage
    strip @Na+
    strip @Cl-
    strip :WAT outprefix dry
    outtraj /user/work/zn22275/{structure}_dry_pdb/run_{index}/dry_md{rep}_0.pdb
    go  
"""    
    with open('initial_pdb.in', "w") as f: 
        f.writelines(initial_pdb)  #write the file

    !cpptraj -i initial_pdb.in  #use cpptraj

list = [100,120,140,160,180,200,220,240,260,280,300,320,340,360,380,400,420,440,460,480]
#apply function
for structure in STRUCTURE:
    for num in range(1,11,1):
        for rep in list:
            initial_pdb(structure=structure,
                        index=num,
                        rep = rep)


CPPTRAJ: Trajectory Analysis. V6.18.1 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 02/06/25 12:31:21
| Available memory: 318.406 GB

INPUT: Reading input from 'initial_pdb.in'
  [parm /bp1/mulholland/zn22275/MalL_RS/output/stripped_MalL_RS.prmtop]
	Reading '/bp1/mulholland/zn22275/MalL_RS/output/stripped_MalL_RS.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin /bp1/mulholland/zn22275/MalL_RS/output_2/run_1/stripped_run_md100.rst7]
	Reading '/bp1/mulholland/zn22275/MalL_RS/output_2/run_1/stripped_run_md100.rst7' as Amber NC Restart
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.
  [strip @Na+]
    STRIP: Stripping atoms in mask [@Na+]
  [strip @Cl-]
    STRIP: Stripping atoms in mask [@Cl-]
  [strip :WAT outprefix dry]
    STRIP: Stripping atoms in mask [:WAT]
	Writing 'stripped' topology file with prefix 'dry'
  [outtraj /user/work/zn22275/RS_dry_pdb/run_1/dry

In [4]:
# 1 sigma analysis Eq - DNEMD
import MDAnalysis as mda
from MDAnalysis.analysis import align
import numpy as np
import matplotlib.pyplot as plt

STRUCTURE= ['RS','TSA']
runs = [1,2,3,4,5,6,7,8,9,10] 
times = [i for i in range(100, 500, 20)]


for structure in STRUCTURE:
# Load the base reference C-alpha structure
    base_ref = mda.Universe(f'/user/work/zn22275/{structure}_dry_pdb/run_1/dry_md100_0.pdb')
    base_ca = base_ref.select_atoms('name CA')

# Create a new Universe object with only the C-alpha atoms
    ca_universe = mda.Merge(base_ca)

# Save the positions of base C-alpha atoms to a file
    np.savetxt("base_ca.txt", base_ca.positions)


    time_points =  [0, 2, 5, 7, 10, 12, 25, 50, 125, 150, 200, 250, 500, 750, 1000, 1250, 2500] 


    for tp in time_points:
        vectors = []
        for run in runs:
            for time in times:
                try:
                    ref = mda.Universe(f'/user/work/zn22275/{structure}_dry_pdb/run_{run}/dry_md{time}_{tp}.pdb')
                    other = mda.Universe(f'/user/work/zn22275/{structure}_dnemd_pdb/run_{run}/dnemd{time}_{tp}.pdb') 
                    mda.analysis.align.alignto(other, ref, select='name CA')
                    ca_ref = ref.select_atoms('name CA').positions
                    ca_other = other.select_atoms('name CA').positions
                    diff = ca_other - ca_ref
                    vectors.append(diff)
                except Exception as e:
                    print(f"Error processing {run}/{time}: {e}")
                    continue

        avg_vectors = np.mean(np.array(vectors), axis=0)

        # Calculate stats
        avg_disp = np.linalg.norm(avg_vectors, axis=1)
        sd = np.std(vectors, axis=0)
        se = sd / np.sqrt(len(vectors))
        avg_x = avg_vectors[:, 0]
        avg_y = avg_vectors[:, 1]
        avg_z = avg_vectors[:, 2]

        # SE of magnitude
        se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))

        adjusted_avg_disp = np.where(np.abs(avg_disp) - se_mag > 0, avg_disp, 0)
        #adjusted_avg_disp = np.where(np.abs(avg_disp) - 2*se_mag > 0, avg_disp, 0)
        adjusted_avg_vectors = np.where(np.abs(avg_vectors) - 2*se > 0, avg_vectors, 0)

        # # Write summary stats
        #     with open(f"vec_norm_stats_{tp}.txt", 'w') as f:
        #         for i in range(len(avg_vectors)):
        #             f.write(f"{adjusted_avg_vectors[i][0]}\t{adjusted_avg_vectors[i][1]}\t{adjusted_avg_vectors[i][2]}\t{adjusted_avg_disp[i]}\t{se[i][0]}\t{se[i][1]}\t{se[i][2]}\t{se_mag[i]}\n")

        # Write summary stats
        with open(f"/user/work/zn22275/dnemd_vec_{structure}/1_sig/dnemd_stats/{structure}_vec_norm_stats_1sig_{tp}.txt", 'w') as f: # change file name
           # Write header
            f.write("#" * 128 + "\n")
            f.write("#                 Average Displacement                Sample Size                                   SE (1SD)                SE (1SD) of              SE (2SD)                SE (2SD) of       #\n")
            f.write("#              --------------------------       ---------------------      Average        --------------------------          Average        --------------------------          Average       #\n")
            f.write("#       CA     x-axis    y-axis    z-axis       x         y         z    Displacement     x-axis    y-axis    z-axis    Displacement vector  x-axis    y-axis    z-axis    Displacement vector #\n")
            f.write("#" * 128 + "\n")

            # Write data
            for i in range(len(avg_vectors)):
                f.write(f"{i+1:>9} {avg_vectors[i][0]:>8.3f} {avg_vectors[i][1]:>8.3f} {avg_vectors[i][2]:>8.3f} ") # removed adjusted vectors
                f.write(f"{len(vectors):>8} {len(vectors):>8} {len(vectors):>8} {avg_disp[i]:>12.3f} ") #removed adjusted displacement
                f.write(f"{se[i][0]:>10.3f} {se[i][1]:>10.3f} {se[i][2]:>10.3f} {se_mag[i]:>12.3f} ")
                f.write(f"{2*se[i][0]:>10.3f} {2*se[i][1]:>10.3f} {2*se[i][2]:>10.3f} {2*se_mag[i]:>12.3f}\n")


        # # Map the norms of the C-alpha vectors onto the C-alpha atoms in the new Universe object
        # for atom, bfactor in zip(ca_universe.atoms, adjusted_avg_disp):
        #     #atom.tempfactors = bfactor
        #     #print(bfactor)
        #     # with open(f"/user/work/zn22275/{structure}_vec/{structure}_vec_norm_{tp}.txt", 'w') as file:
        #     #     file.write(bfactor)

        # # ca_universe.atoms.tempfactors = adjusted_avg_disp
    
        # # Write the C-alpha-only structure with mapped B-factors
        # pdb_filename = f'/user/work/zn22275/{structure}_vec/{structure}_vec_norm_{tp}.pdb'
        # ca_universe.atoms.write(pdb_filename)

        with mda.Writer("temp_ca.pdb", ca_universe.atoms.n_atoms) as w: 
            w.write(ca_universe)
   
        # Now read it back and set the B-factors
        new_universe = mda.Universe("temp_ca.pdb")
        new_universe.add_TopologyAttr('tempfactors')  # Add the B-factor attribute explicitly
        new_universe.atoms.tempfactors = adjusted_avg_disp
   
        # Write the final PDB with B-factors
        pdb_filename = f'/user/work/zn22275/dnemd_vec_{structure}/1_sig/{structure}_vec_norm_1sig_{tp}.pdb'
        with mda.Writer(pdb_filename) as w:
            w.write(new_universe)


/tmp/ipykernel_2251709/2975538709.py:54: RuntimeWarning: divide by zero encountered in divide
  se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))
/tmp/ipykernel_2251709/2975538709.py:54: RuntimeWarning: invalid value encountered in multiply
  se_mag = (1 / avg_disp) * np.sqrt((avg_x**2)*(se[:, 0]**2) + (avg_y**2)*(se[:, 1]**2) + (avg_z**2)*(se[:, 2]**2))
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:334: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/user/work/zn22275/anaconda3/envs/dnemd/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
